In [2]:
import os
import requests
import base64
import json
from dotenv import load_dotenv
load_dotenv()

def get_spotify_access_token():
    """
    Retrieves an access token from Spotify using the Client Credentials Flow.
    Make sure that SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET are set in your environment.
    """
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

    if not client_id or not client_secret:
        raise Exception("Spotify credentials not found in environment variables.")

    # Create a Base64-encoded string from 'client_id:client_secret'
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()

    token_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {b64_auth_str}",
        "Content-type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code != 200:
        raise Exception(f"Failed to get access token: {response.text}")

    token = response.json()["access_token"]
    return token

In [6]:
def search_song_by_title(song_recommendation, token):
    """
    Searches Spotify for a track matching the song recommendation.
    Returns a dictionary with song title, artist(s), and a Spotify link.
    """
    search_url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    params = {
        "q": song_recommendation,
        "type": "track",
        "limit": 1
    }

    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code != 200:
        print("Error searching Spotify", response.text)
        return None

    data = response.json()
    tracks = data.get("tracks", {}).get("items", [])
    if not tracks:
        print("No tracks found for the recommendation:", song_recommendation)
        return None

    # Take the first track as the best match
    track = tracks[0]
    song_title = track.get("name")
    
    # Get a comma-separated list of artist names
    artists = [artist.get("name") for artist in track.get("artists", [])]
    spotify_link = track.get("external_urls", {}).get("spotify")

    # Get the album cover art (has 3 sizes: large, medium, small)
    album_images = track.get("album", {}).get("images", [])
    album_art_url = album_images[0]['url'] if album_images else None  # Highest resolution

    return {
        "song_title": song_title,
        "artist": ", ".join(artists),
        "spotify_link": spotify_link,
        "album_art": album_art_url
    }

In [7]:
# Testing the function

# 1st, get an access token using the helper func
token = get_spotify_access_token()
print("Access token acquired:", token[:20] + "...") # print a snippet for confirmation

# Sample song to test
test_song = "Шүрэн бугуйвч"

song_details = search_song_by_title(test_song, token)

if song_details:
    print("Song details:")
    print(json.dumps(song_details, indent=2))
else:
    print("No song details were found for:", test_song)

Access token acquired: BQDwPWc_Pm7pDu5yjL-S...
Song details:
{
  "song_title": "\u0428\u04af\u0440\u044d\u043d \u0411\u0443\u0433\u0443\u0439\u0432\u0447",
  "artist": "Bayartsetseg, Amraa",
  "spotify_link": "https://open.spotify.com/track/6ZSCn0ZFsmRLsNvBHOihP1",
  "album_art": "https://i.scdn.co/image/ab67616d0000b2739344c68be65bc577694548b3"
}


In [8]:
# Sample song to test
test_song = "Bohemian Rapsdy"

song_details = search_song_by_title(test_song, token)

if song_details:
    print("Song details:")
    print(json.dumps(song_details, indent=2))
else:
    print("No song details were found for:", test_song)

Song details:
{
  "song_title": "Bohemian Rhapsody - Remastered 2011",
  "artist": "Queen",
  "spotify_link": "https://open.spotify.com/track/7tFiyTwD0nx5a1eklYtX2J",
  "album_art": "https://i.scdn.co/image/ab67616d0000b273ce4f1737bc8a646c8c4bd25a"
}


In [9]:
# Sample song to test
test_song = "buy u a drink"

song_details = search_song_by_title(test_song, token)

if song_details:
    print("Song details:")
    print(json.dumps(song_details, indent=2))
else:
    print("No song details were found for:", test_song)

Song details:
{
  "song_title": "Buy U a Drank (Shawty Snappin') (feat. Yung Joc)",
  "artist": "T-Pain, Yung Joc",
  "spotify_link": "https://open.spotify.com/track/1tkMs9wDPknQ3ZHcKDw1ES",
  "album_art": "https://i.scdn.co/image/ab67616d0000b273baf2f329d4feeb19761eddf4"
}
